# Look at accelerometer data 

Finding Zero velocity times by rail axis acceleration noise levels, making summary statistics for the noise levels across the whole day files.  Spot check graphs to see what works

In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp

#Custome graph format style sheet
#plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'

#Standard cycle to make black and white images and dashed and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '--', ':', '-.']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')

In [2]:
#Extra Headers:
import os as os
import pywt as py
import statistics as st
import os as os
import random
import multiprocessing
from joblib import Parallel, delayed
import platform

from time import time as ti

In [3]:
import CoreFunctions as cf
from skimage.restoration import denoise_wavelet

## Choosing Platform
Working is beinging conducted on several computers, and author needs to be able to run code on all without rewriting..  This segment of determines which computer is being used, and sets the directories accordingly.

In [4]:
HostName = platform.node()

if HostName == "Server":
    Computer = "Desktop"   
elif HostName[-6:] == 'wm.edu':
    Computer = "SciClone"
elif HostName == "SchoolLaptop":
    Computer = "LinLap"
elif HostName == "WTC-TAB-512":
    Computer = "PortLap"
else:
    Computer = "WinLap"

In [5]:
if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/image/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'
elif Computer == 'PortLap':
    location = 'C:\\users\\dhendrickson\\Desktop\\AccelData\\'

In [6]:
if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    folder = '/sciclone/scr10/dchendrickson01/Recordings2/'
elif Computer == "Desktop":
    rootfolder = location
    folder = rootfolder + "Recordings2\\SubSet\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'Recordings2/'
elif Computer =='PortLap':
    rootfolder = location 
    folder = rootfolder + 'Recordings2\\'

In [7]:
files = []
files.append('220919 recording3.csv')
files.append('230130 recording4.csv')
files.append('230418 recording2.csv')

In [8]:
Saving = False
location = folder
Titles = True
Ledgends = True

gdpi=2400


In [9]:
def RollingStdDev(RawData, SmoothData, RollSize = 25):
    StdDevs = []
    for i in range(RollSize):
        Diffs = RawData[0:i+1]-SmoothData[0:i+1]
        Sqs = Diffs * Diffs
        Var = sum(Sqs) / (i+1)
        StdDev = np.sqrt(Var)
        StdDevs.append(StdDev)
    for i in range(len(RawData)-RollSize-1):
        j = i + RollSize
        Diffs = RawData[i:j]-SmoothData[i:j]
        Sqs = Diffs * Diffs
        Var = sum(Sqs) / RollSize
        StdDev = np.sqrt(Var)
        StdDevs.append(StdDev)  
    
    return StdDevs

def RollingSum(Data, Length = 100):
    RollSumStdDev = []
    for i in range(Length):
        RollSumStdDev.append(sum(Data[0:i+1]))
    for i in range(len(Data) - Length):
        RollSumStdDev.append(sum(Data[i:i+Length]))
    return RollSumStdDev

def SquelchPattern(DataSet, StallRange = 5000, SquelchLevel = 0.0086):
    SquelchSignal = []

    for i in range(StallRange):
        SquelchSignal.append(1)

    for i in range(len(DataSet)-2*StallRange):
        
        if np.average(DataSet[i:i+2*StallRange]) < SquelchLevel:
            SquelchSignal.append(0)
        else:
            SquelchSignal.append(1)

    for i in range(StallRange+1):
        SquelchSignal.append(1)    
    
    return SquelchSignal

def getVelocity(Acceleration, Timestamps = 0.003, Squelch = [], corrected = 0):
    velocity = np.zeros(len(Acceleration))
    
    if Squelch == []:
        Squelch = np.ones(len(Acceleration))
    
    if len(Timestamps) == 1:
        dTime = Timestamps[0]
    elif len(Timestamps) == len(Acceleration):
        totTime = 0
        for i in range(len(Timestamps)-1):
            if Timestamps[i]<Timestamps[i+1]:
                totTime += (Timestamps[i+1] - Timestamps[i])
            else:
                totTime += (Timestamps[i+1] - Timestamps[i] + 10000)
        dTime = totTime / len(Timestamps)
    else:
        print('error')

    dTime = dTime / 10000.0

    velocity[0] = Acceleration[0] * (dTime)
    
    MoveStart = 0
    for i in range(len(Acceleration)-1):
        j = i + 1
        if corrected ==2:
            if Squelch[j]==0:
                velocity[j]=0
                MoveStart = j
            else:
                velocity[j] = velocity[i] + Acceleration[j] * dTime 
                try:
                    if Squelch[j+1]  == 0:
                        PointVairance = velocity[j] / (j - MoveStart)
                        for k in range(j-MoveStart):
                            velocity[k+MoveStart] -=  PointVairance * k
                except:
                    pass
        else:
            velocity[j] = velocity[i] + Acceleration[j] * dTime

    if corrected == 1:
        PointVairance = velocity[-1:] / len(velocity)
        for i in range(len(velocity)):
            velocity[i] -=  PointVairance * i
    
    velocity *= 9.81

    return velocity



In [10]:
#Smooth = cf.Smoothing(ODataSet[:,3],2) #,50)
def DeviationVelocity(file):
    if file[-3:] =='csv':
        #try: 
        ODataSet = np.genfromtxt(open(folder+file,'r'), delimiter=',',skip_header=0,missing_values=0,invalid_raise=False)
        SmoothX = denoise_wavelet(ODataSet[:,3], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        SmoothY = denoise_wavelet(ODataSet[:,4], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        SmoothZ = denoise_wavelet(ODataSet[:,5], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        StdDevsX = RollingStdDev(ODataSet[:,3],SmoothX)
        StdDevsY = RollingStdDev(ODataSet[:,4],SmoothY)
        StdDevsZ = RollingStdDev(ODataSet[:,5],SmoothZ)
        StdDevsX.append(0)
        StdDevsY.append(0)
        StdDevsZ.append(0)
        StdDevsX = np.asarray(StdDevsX)
        StdDevsY = np.asarray(StdDevsY)
        StdDevsZ = np.asarray(StdDevsZ)
        SmoothDevX = denoise_wavelet(StdDevsX, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        SmoothDevY = denoise_wavelet(StdDevsY, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        SmoothDevZ = denoise_wavelet(StdDevsZ, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        #RollSumStdDevX = RollingSum(SmoothDevX)
        #RollSumStdDevX = np.asarray(RollSumStdDevX)
        #RollSumStdDevY = RollingSum(SmoothDevY)
        #RollSumStdDevY = np.asarray(RollSumStdDevY)
        #RollSumStdDevZ = RollingSum(SmoothDevZ)
        #RollSumStdDevZ = np.asarray(RollSumStdDevZ)
        Max = np.max(SmoothDevX)
        buckets = int(Max / 0.01) + 1
        bins = np.linspace(0,buckets*0.01,buckets+1)
        counts, bins = np.histogram(SmoothDevX,bins=bins)
        SquelchLevel = bins[np.argmax(counts)+1] 
        SquelchSignal = SquelchPattern(SmoothDevX, 5000, SquelchLevel)
        SmoothX -= np.average(SmoothX)
        Velocity = getVelocity(SmoothX, ODataSet[:,2],SquelchSignal, 2)
        Velocity = np.asarray(Velocity)

        '''except:
            Velocity = file
            StdDevsX = 0
            SmoothDevX = 0
            StdDevsY = 0
            SmoothDevY = 0
            StdDevsZ = 0
            SmoothDevZ = 0
            SquelchSignal=0
            SmoothX=0
            ODataSet=np.zeros((5,5))
            print(file)
        '''
        return [Velocity, [SmoothDevX, SmoothDevY, SmoothDevZ], [StdDevsX,StdDevsY,StdDevsZ], SquelchSignal,[SmoothX, SmoothY, SmoothZ],ODataSet[:,3],ODataSet[:,1]]
    else:
        pass

In [11]:
def getSecondsFromString(Value):
    Hour = int(Value[0:2])
    Minute = int(Value[2:4])
    Seconds = int(Value[4:6])
    
    Total = (Hour * 3600) + (Minute * 60) + Seconds

    return Total

In [12]:
def getFixedTime(file):
    ODataSet = np.genfromtxt(open(file,'r'), delimiter=',',skip_header=0,missing_values=0,invalid_raise=False)
    
    Timings = []
    lastmicrosecond = 0
    CurrentSecond = 0
    
    for row in ODataSet:
        if row[2] < lastmicrosecond:
            CurrentSecond+=1
        lastmicrosecond = row[2]
        
        Hour = int(CurrentSecond / 3600)
        Minute = int((CurrentSecond - (Hour * 3600)) / 60)
        Second = CurrentSecond - (Hour * 3600) - (Minute * 60)
        
        Timings.append(str(Hour).zfill(2)+str(Minute).zfill(2) + str(Second).zfill(2))
        
    return Timings

In [13]:
#files = fi2

In [14]:
LoopFiles = 12
loops = int(len(files) / LoopFiles) +1
bump = 0

In [15]:
for j in range(loops-bump):
    k = j + bump
    if k == loops -1:
        tfiles = files[k*LoopFiles:]
    else:
        tfiles = files[k*LoopFiles:(k+1)*LoopFiles]
    Results = []
    #for file in files:
    #    result = DeviationVelocity(file)
    #    Results.append(result)
    Results = Parallel(n_jobs=LoopFiles)(delayed(DeviationVelocity)(file) for file in tfiles)
    Velocities = []
    StdDevsX = []
    SumStdDevsX = []
    StdDevsY = []
    SumStdDevsY = []
    StdDevsZ = []
    SumStdDevsZ = []
    NewFiles=[]
    SquelchSignals=[]
    SmoothX=[]
    SmoothY=[]
    SmoothZ=[]
    RwSignals=[]
    Timings=[]

    for i in range(len(Results)):
        if Results[i] != None:
            Velocities.append(Results[i][0])
            StdDevsX.append(Results[i][1][0])
            SumStdDevsX.append(Results[i][2][0])
            StdDevsY.append(Results[i][1][1])
            SumStdDevsY.append(Results[i][2][1])
            StdDevsZ.append(Results[i][1][2])
            SumStdDevsZ.append(Results[i][2][2])
            SquelchSignals.append(Results[i][3])
            SmoothX.append(Results[i][4][0])
            SmoothY.append(Results[i][4][1])
            SmoothZ.append(Results[i][4][2])
            RwSignals.append(Results[i][5])
            Timings.append(Results[i][6])
            NewFiles.append(tfiles[i])
            

In [23]:
f=2

In [24]:
try:
    Seconds = getSecondsFromString(str(Timings[f][-3])) - getSecondsFromString(str(Timings[f][1]))
except:
    CheckT = getFixedTime(folder+NewFiles[f])
    Timings[f] = CheckT
    Seconds = getSecondsFromString(str(Timings[f][-3])) - getSecondsFromString(str(Timings[f][1]))

In [25]:
Seconds = getSecondsFromString(str(Timings[f][-3])) - getSecondsFromString(str(Timings[f][1]))
TotalTime = np.linspace(0,Seconds/3600, np.shape(Timings[f])[0])

In [26]:
TempSig = SmoothX[f][:] - np.average(SmoothX[f][:])
Vs = getVelocity(TempSig,[33],[],0)

In [27]:
fig, ax = plt.subplots(1,1,figsize=(5,3),dpi=gdpi,linewidth=0.0625)
ax.plot(TotalTime[:], Vs )
ax.set_xlabel('Hours')
ax.set_ylabel('m/s')

Text(0, 0.5, 'm/s')

In [28]:
fig, ax = plt.subplots(1,1,figsize=(5,3),dpi=gdpi,linewidth=0.0625)
ax.plot(TotalTime[:6000000], Vs[:6000000] )
ax.set_xlabel('Hours')
ax.set_ylabel('m/s')

Text(0, 0.5, 'm/s')

In [29]:
fig, ax = plt.subplots(1,1,figsize=(5,3),dpi=gdpi,linewidth=0.0625)
ax.plot(TotalTime[:6000000], Velocities[f][:6000000] )
ax.set_xlabel('Hours')
ax.set_ylabel('m/s')

Text(0, 0.5, 'm/s')

In [ ]:
## for f=0; '220919 recording3.csv'
begin=1500000
finish=begin + 100000

## for f = 1; '230131 recording4.csv'
#begin = 14000000
#finish = begin + 500000
## '230130 recording4.csv'
#begin = 3500000
#finish = begin + 500000

## for f =2
#begin = 12000000
#finish = begin + 250000
# 'something18'
#begin=4990000
#finish=begin+55000

start = getSecondsFromString(str(Timings[f][finish])) 
end = getSecondsFromString(str(Timings[f][begin]))
TotalTime = np.linspace(start/60,end/60, finish-begin)

In [ ]:
vs = getVelocity(SmoothX[f][begin:finish],[33],[],1)


In [ ]:
fig, ax1 = plt.subplots(figsize=(6.67,3.75),dpi=gdpi,linewidth=0.125) 
  
ax1.set_ylabel('Acceleration (g)', color = 'grey') 
plt.ylim(-0.4,0.4)
ax1.plot(TotalTime, SmoothX[f][begin:finish], color = 'grey') 
ax1.tick_params(axis ='y', labelcolor = 'grey') 
ax1.set_xlabel('minutes')
 
# Adding Twin Axes

ax2 = ax1.twinx() 

ax2.set_xlabel('minutes') 
ax2.set_ylabel('Velocity (m/s)', color = 'red') 
plt.ylim(-6,6)
ax2.plot(TotalTime, Velocities[f][begin:finish], color = 'red', linestyle = 'dashed' )
ax2.tick_params(axis ='y', labelcolor = 'red') 

# Show plot

plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(6.67,3.75),dpi=gdpi,linewidth=0.125) 
  
ax1.set_ylabel('Acceleration (g)', color = 'grey') 
plt.ylim(-0.4,0.4)
ax1.plot(TotalTime, SmoothX[f][begin:finish], color = 'grey') 
ax1.tick_params(axis ='y', labelcolor = 'grey') 
ax1.set_xlabel('minutes')
 
# Adding Twin Axes

ax2 = ax1.twinx() 

ax2.set_xlabel('minutes') 
ax2.set_ylabel('Velocity (m/s)', color = 'red') 
plt.ylim(-6,6)
ax2.plot(TotalTime, Velocities[f][begin:finish], color = 'red', linestyle = 'dashed' )
ax2.tick_params(axis ='y', labelcolor = 'red') 

# Show plot

plt.show()

In [ ]:
len(Vs)

In [ ]:
fig, ax1 = plt.subplots(figsize=(6.67,3.75),dpi=gdpi,linewidth=0.125) 


ax1.set_ylabel('Acceleration (g)', color = 'grey') 
ax1.plot(TotalTime, SmoothX[f][begin:finish], color = 'grey', label='Acceleration') 
ax1.tick_params(axis ='y', labelcolor = 'grey') 
plt.ylim(-0.4,0.4)

# Adding Twin Axes
ax2 = ax1.twinx() 

ax2.set_xlabel('Time') 
ax2.set_ylabel('RSD - $V$', color = 'blue') 
ax2.plot(TotalTime, StdDevsZ[f][begin:finish], color = 'blue', linestyle = 'dashed', label='RSD - $V$' )
ax2.tick_params(axis ='y', labelcolor = 'blue') 
plt.ylim(-0.005,0.1)
ax2.set_xlabel('Minutes')

# Show plot

plt.show()

In [ ]:
fig, ax1 = plt.subplots(3,1,figsize=(6.67,6.67),dpi=gdpi,linewidth=0.125) 
ax1[0].set_ylabel('RSD $V$', color = 'blue') 
ax1[0].plot(TotalTime, StdDevsZ[f][begin:finish], color = 'blue', linestyle = 'dashed', label='RSD $V$' )
ax1[0].tick_params(axis ='y', labelcolor = 'blue') 
ax1[0].set_ylim(-0.005,0.1)

ax1[1].set_ylabel('RSD $H$', color = 'blue') 
ax1[1].plot(TotalTime, StdDevsY[f][begin:finish], color = 'blue', linestyle = 'dashed', label='RSD $H$' )
ax1[1].tick_params(axis ='y', labelcolor = 'blue') 
ax1[1].set_ylim(-0.005,0.1)

ax1[2].set_ylabel('RSD $P$', color = 'blue') 
ax1[2].plot(TotalTime, StdDevsX[f][begin:finish], color = 'blue', linestyle = 'dashed', label='RSD $P$' )
ax1[2].tick_params(axis ='y', labelcolor = 'blue') 
ax1[2].set_ylim(-0.005,0.1)

ax1[2].set_xlabel('Minutes')


In [ ]:
fig, ax1 = plt.subplots(figsize=(6.67,3.75),dpi=gdpi,linewidth=0.125) 


ax1.set_ylabel('RSD - $V$', color = 'blue') 
ax1.plot(TotalTime, StdDevsZ[f][begin:finish], color = 'blue', label='Std Dev of Acceleration') 
ax1.tick_params(axis ='y', labelcolor = 'blue') 
plt.ylim(-0.005,0.1)


# Adding Twin Axes
ax2 = ax1.twinx() 

ax2.set_xlabel('Time') 
ax2.set_ylabel('Speed', color = 'red') 
ax2.plot(TotalTime, np.abs(Velocities[f][begin:finish]), color = 'red', linestyle = 'dashed', label='Velocity' )
ax2.tick_params(axis ='y', labelcolor = 'red') 
plt.ylim(-0.,6)
ax2.set_xlabel('Minutes')

# Show plot

plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(6.67,3.75),dpi=gdpi,linewidth=0.125) 

ax1.set_ylabel('RSD - $V$', color = 'blue') 
ax1.plot(TotalTime, StdDevsZ[f][begin:finish], color = 'blue', label='Std Dev of Acceleration') 
ax1.tick_params(axis ='y', labelcolor = 'blue') 
plt.ylim(-0.005,0.1)
# Adding Twin Axes
ax2 = ax1.twinx() 

ax2.set_ylabel('Squelch Signal', color = 'green') 
ax2.plot(TotalTime, SquelchSignals[f][begin:finish], color = 'green', label='Std Dev of Acceleration') 
ax2.tick_params(axis ='y', labelcolor = 'green') 
plt.ylim(-0.1,1.1)
ax2.set_xlabel('Minutes')

# Show plot

plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(6.67,3.75),dpi=gdpi,linewidth=0.125) 

ax1.set_ylabel('Acceleration (g)', color = 'grey') 
ax1.plot(TotalTime, SmoothX[f][begin:finish], color = 'grey', label='Acceleration') 
ax1.tick_params(axis ='y', labelcolor = 'grey') 
plt.ylim(-0.4,0.4)
# Adding Twin Axes
ax2 = ax1.twinx() 

ax2.set_xlabel('Time') 
ax2.set_ylabel('Squelch Signal', color = 'green') 
ax2.plot(TotalTime, SquelchSignals[f][begin:finish], color = 'green', linestyle = 'dashed', label='Velocity' )
ax2.tick_params(axis ='y', labelcolor = 'green') 
ax2.set_xlabel('Minutes')
plt.ylim(-0.1,1.1)


# Show plot

plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(6.67,3.75),dpi=gdpi,linewidth=0.125) 

ax1.set_ylabel('Squelch Signal', color = 'green') 
ax1.plot(TotalTime, SquelchSignals[f][begin:finish], color = 'green', label='Std Dev of Acceleration') 
ax1.tick_params(axis ='y', labelcolor = 'green') 
plt.ylim(-0.1,1.1)

# Adding Twin Axes
ax2 = ax1.twinx() 

ax2.set_xlabel('Time') 
ax2.set_ylabel('Velocity', color = 'red') 
ax2.plot(TotalTime, Velocities[f][begin:finish], color = 'red', linestyle = 'dashed', label='Velocity' )
ax2.tick_params(axis ='y', labelcolor = 'red') 
ax2.set_xlabel('Minutes')
plt.ylim(-6,6)

# Show plot

plt.show()

In [ ]:
asdfasdf

In [ ]:
Seconds = getSecondsFromString(str(Timings[f][-3])) - getSecondsFromString(str(Timings[f][1]))
TotalTime = np.linspace(0,Seconds/3600, np.shape(Timings[f])[0])

In [ ]:
for RSD in StdDevsX:
    Max = np.max(RSD)
    buckets = int(Max / 0.01) + 1
    bins = np.linspace(0,buckets*0.01,buckets+1)
    fig,ax = plt.subplots()
    ax.hist(RSD,bins=bins)
    counts, bins = np.histogram(RSD,bins=bins)
    print(counts,bins)
    print(bins[np.argmax(counts)])
    plt.show()

In [ ]:
for d in StdDevsX:
    fig = plt.figure()
    plt.plot(d)
    plt.show()

In [ ]:
for i in range(10):
    print(i)
    i+=1
